
#  Fine-tuning of Alpaca 🦙 on French




#Dependence

In [1]:
#Preparing the environement
!pip install -q datasets 
!pip install -q loralib
!pip install -q sentencepiece
!pip install -q git+https://github.com/huggingface/transformers.git
# transformers from source
!pip install git+https://github.com/huggingface/transformers -q
!pip install -q accelerate
!pip install -q bitsandbytes
!pip install -q git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from datasets import load_dataset
import pandas as pd
import numpy as np
from datasets import Dataset, load_dataset
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model
import bitsandbytes as bnb
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-t4-s-3373k6jj1auxe --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required=

## Concatenate the data

In [3]:
# pandas load 3 xlsx files
df1 = pd.read_excel("/content/alpaca_data1_fr.xlsx")
df2 = pd.read_excel("/content/alpaca_data2_fr.xlsx")
df3 = pd.read_excel("/content/alpaca_data3_fr.xlsx")

# concat 3 xlsx files
df = pd.concat([df1, df2, df3])

# PUSH THE DATASET TO THE HUB

In [8]:
dataset = Dataset.from_pandas(df)

In [9]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [10]:
dataset.push_to_hub("tbboukhari/Alpaca_french_instruct")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/53 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
dataset = load_dataset("tbboukhari/Alpaca_french_instruct")

  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
dataset['train'][0]

{'instruction': ' Donnez trois conseils pour rester en bonne santé.',
 ' saisir': '',
 ' sortir': " 1. Ayez une alimentation équilibrée et assurez-vous d'inclure beaucoup de fruits et de légumes. \\n2. Faites de l'exercice régulièrement pour garder votre corps actif et fort. \\n3. Dormez suffisamment et maintenez un horaire de sommeil constant.",
 '__index_level_0__': 0}

In [5]:
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model
import bitsandbytes as bnb
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-t4-s-3373k6jj1auxe --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required=

In [7]:
MICRO_BATCH_SIZE = 2  # this could actually be 5 but i like powers of 2
BATCH_SIZE = 128
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 1  # we don't need 3 tbh
LEARNING_RATE = 3e-4  # the Karpathy constant
CUTOFF_LEN = 256  # 256 accounts for about 96% of the data
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

In [8]:
load_in_8bit_fp32_cpu_offload=True
model = LlamaForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
tokenizer = LlamaTokenizer.from_pretrained(
    "decapoda-research/llama-7b-hf", add_eos_token=True
)

model = prepare_model_for_int8_training(model)

config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
) 
tokenizer.pad_token_id = 0
model = get_peft_model(model, config)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


## Preprocess

In [23]:
def generate_prompt(data_point):
    # desculpe o desastre de formatação, preciso ser rápido
    if data_point[" saisir"]:
        return f"""Vous trouverez ci-dessous des instructions décrivant une tâche, ainsi qu'une entrée qui fournit plus de contexte. Rédigez une réponse qui complète convenablement la demande.
### instructions:
{data_point["instruction"]}
### entrée:
{data_point[" saisir"]}
### sortie:
{data_point[" sortir"]}"""
    else:
        return f"""Vous trouverez ci-dessous des instructions décrivant une tâche, ainsi qu'une entrée qui fournit plus de contexte. Rédigez une réponse qui complète convenablement la demande.
### directions:
{data_point["instruction"]}
### sortie:
{data_point[" sortir"]}"""



def tokenize(prompt):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN + 1,
        padding="max_length",
    )
    return {
        "input_ids": result["input_ids"][:-1],
        "attention_mask": result["attention_mask"][:-1],
    }

In [24]:
dataset = dataset.shuffle().map(lambda x: tokenize(generate_prompt(x)))

Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

## Training

In [3]:
import transformers


In [26]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=100,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=20,
        output_dir="lora-alpaca",
        save_total_limit=3,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train(resume_from_checkpoint=False)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
20,2.009900
40,1.518900
60,0.945600
80,0.856200
100,0.830600
120,0.816900


KeyboardInterrupt: ignored

In [27]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [28]:
model.push_to_hub("tbboukhari/Alpaca_instruction_fine_tune_French", use_auth_token=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/tbboukhari/Alpaca_instruction_fine_tune_French/commit/c14f62b5aa91b50b36be1a46c9af67cf2f4a44ee', commit_message='Upload model', commit_description='', oid='c14f62b5aa91b50b36be1a46c9af67cf2f4a44ee', pr_url=None, pr_revision=None, pr_num=None)

In [4]:
import torch
import bitsandbytes as bnb
from peft import PeftModel, PeftConfig, prepare_model_for_int8_training, LoraConfig, get_peft_model
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig

peft_model_id = "tbboukhari/Alpaca_instruction_fine_tune_French"
# config = PeftConfig.from_pretrained(peft_model_id)

tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = LlamaForCausalLM.from_pretrained("decapoda-research/llama-7b-hf",
                                          load_in_8bit=True,
                                          device_map="auto",)
# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [7]:

def generate_prompt(instruction, entrée=None):
    if entrée :
        return f"""Vous trouverez ci-dessous des instructions décrivant une tâche, ainsi qu'une entrée qui fournit plus de contexte. Rédigez une réponse qui complète convenablement la demande.
### instructions:
{instruction}
### entrée:
{entrée}
### sortie:"""

    else:
        return f"""Vous trouverez ci-dessous des instructions décrivant une tâche, ainsi qu'une entrée qui fournit plus de contexte. Rédigez une réponse qui complète convenablement la demande.

### instructions:
{instruction}
### sortie:"""




In [8]:
generation_config = GenerationConfig(
    temperature=0.2,
    top_p=0.75,
    num_beams=4,
)

def evaluate(instruction, input=None):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        print("sortie:", output.split("### sortie:")[1].strip())

In [15]:
evaluate(input("instructions: "))

instructions: qui est le roi actuel de la grande bretagne
sortie: Le roi actuel de la Grande-Bretagne est le roi Édouard VII.
